<img src="images/inmas.png" width=130x align='right' />

# Solutions to Exercise 16 - More Advanced NumPy


### Prerequisite
Notebook 16

### 1.
Using the following data

In [ ]:
data = np.genfromtxt('./data/stockholm_td_adj.csv', delimiter=',', skip_header=1)
months = ['January', 'February', 'March',
          'April', 'May', 'June',
          'July', 'August', 'September',
          'October', 'November', 'December',
         ]

1.   Extract the mean monthly average temperatures for each month of the year.
2.   Plot the extracted mean values by months, with the proper month labeling.

---------------------------------
<font face="verdana" style="font-size:20px" color="blue"> Solution 1. </font>   


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

data = np.genfromtxt('../data/stockholm_td_adj.csv', delimiter=',', skip_header=1)
months = ['January', 'February', 'March',
          'April', 'May', 'June',
          'July', 'August', 'September',
          'October', 'November', 'December',
         ]

# Handle fractional year correctly (from previous exercise)
fracyear = np.zeros_like(data[:,0])
for i in range(len(data[:, 0])):
    dayoftheyear = datetime(int(data[i, 0]), int(data[i, 1]), int(data[i, 2])).timetuple().tm_yday
    # Divide by December 31st of that year to account for leap years
    fracyear[i] = data[i, 0] + dayoftheyear/datetime(int(data[1, 0]), 12, 31).timetuple().tm_yday

mdata = np.vstack((fracyear, data[:,1], data[:,3]))
print(mdata.shape)
fig, ax = plt.subplots(figsize=(14,4))
for m in range(12):
    mask = mdata[1,:] == m
    ax.plot(mdata[0, mask], mdata[2, mask], label=months[m])
    
ax.axis('tight')
ax.set_title('temperatures in Stockholm')
ax.set_xlabel('year')
ax.set_ylabel('temperature (C)')
ax.legend();

### 2.*
__Newton's method__

This problem has three parts: 

* __2.1__: [Newton's Method](https://en.wikipedia.org/wiki/Newton%27s_method) is a root finding method that uses linear approximation. In particular, we guess a solution $x_0$ of the equation $f(x)= 0$, compute the linear approximation of $f(x)$ at $x_0$ and then find the x-intercept of the linear approximation. The general scheme of Newton's method may be written as: 
$$ x_{n+1} = x_n - \dfrac{f(x_n)}{f'(x_n)} $$ for $n=0,1, 2, ...$. The computation is repeated until $f(x_n)$ is close enough to zero. More precisely, we test if $|f(x_n)|< \epsilon$ , with $\epsilon$ being a small number. Write a function that returns an approximation of a solution of  $f(x)=0$ by Newton's method. Make sure that your function prints out an array containing the approximations as a function of iterations. 
    
* __2.2__: Let $f(x)= x^2- 9$. Using the function from Part 1, calculate an approximation for the solution of $f(x)=0$ using $x_0= 1000$ and $\epsilon= 0.01$. Make a convergence plot of the approximate solution as a function of iteration. Make this plot a scatter plot with scales log linear (x axis linear, y axis logarithmic).
    
* __2.3__: Create a Figure with two subplots, one next to the other. 
    * First plot: Plot the function $f(x)$ using `x=np.arange(0, 10)`. You should label where the solution to $f(x)=0$ is at, i.e. label the point $(3,0)$. Draw a horizontal line at $y =0$, you can use `plt.hlines()`.
    * Second plot: Plot the array from Part 2. 
    
    Make sure each graph has a title, labels, and correct limits.

---------------------------------
<font face="verdana" style="font-size:20px" color="blue"> Solution 2. </font>   


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    '''Function to find zero.'''
    return x**2 - 9 


def df(x):
    '''Derivative of function.'''
    return 2*x


def newton(f, df, x0, eps):
    '''Newton\'s method for root finding.'''
    mylist = []
    x = x0
    it = 0
    while abs(f(x)) > eps:
        x = x - f(x)/df(x)
        mylist.append(x)
        it += 1
    return it, mylist


niter, values = newton(f, df, 1000, 0.01)

# Create figure and subplots
x = np.arange(0, 10, 1)
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))

# plot of first graph
ax1.plot(x , f(x), color= 'purple')
ax1.axhline(y=0, color='grey', linestyle=':')

# Title, x and y labels, x and y limits
ax1.set_title('Plot of $f(x) = x^2 -9$')
ax1.set_ylabel('$f(x)= x^2 -9$')
ax1.set_xlabel('x')
ax1.set_xlim(-1, 10)
ax1.set_ylim(-10, 100)
ax1.annotate('/__  (3,0)', (3,0))

# plot of second graph
ax2.scatter(range(1, niter+1), values, color='green', marker='*')
ax2.set_yscale('log')
ax2.set_title('Convergence of Solution')
ax2.set_ylabel('Approximation') 
ax2.set_xlabel('Number of Iterations')
ax2.set_xlim(0, 11)
# ax2.axhline(y=0, color='gray', linestyle=':');
# ax2.set_ylim(0, 500);

### 3.*

__Euler's method for solving ordinary differential equation__

For a first order ordinary differential equation, 
$$\frac{dS(t)}{dt} = F(t, S(t))$$
the linear approximation of $S(t)$ around $t_j$ at $t_{j+1}$ is called the *Explicit Euler Formula* :
$$ S(t_{j+1})= S(t_j) + hF(t_j, S(t_j))$$
where $h = t_{j+1} - t_j$ is a stepsize of the interval $[t_0, t_f]$.
Given initial value of $S_0 = S(t_0)$, we can use the formula to integrate the states up to $S(t_f)$. These $S(t)$ values are an approximated solution of the differential equation.

Consider the differential equation 
$$\frac{df(t)}{dt} = e^{-t} $$ with initial condition $f(0)=−1$.  Approximate the solution to this initial value problem between 0 and 1 in increments of 0.01 using the Euler's method. Plot the approximated solution with the exact solution. Note that the exact solution is $f(t)=-e^{-t}$.

---------------------------------
<font face="verdana" style="font-size:20px" color="blue"> Solution 3. </font>   


In [ ]:
# Define ODE
f = lambda t, s: np.exp(-t) 

# Step size
h = 0.01 
t = np.arange(0, 1 + h, h) 

# Solutions using Euler Method
S = np.zeros(len(t))

# Initial Conditions
S[0] = -1

# Easier to use a loop in cases where values have different indices 
for i in range(0, len(t) - 1):
    S[i + 1] = S[i] + h*f(t[i], S[i])

plt.figure(figsize = (7, 5))
plt.plot(t, S, 'b', label='Approximate')
plt.plot(t, -np.exp(-t), 'r--', label='Exact')
plt.title('Approximate and Exact Solutions for $f(t) = e^{-t}, f(0) = -1$')
plt.xlabel('$t$')
plt.ylabel('$f(t)$')
plt.legend(loc='best');

#### 4.
Can the transpose of a matrix be obtained through the `reshape` method? Explain.

---------------------------------
<font face="verdana" style="font-size:20px" color="blue"> Solution 4. </font>   
